In [ ]:
from main import RF
from mmdit import MMDiT_for_IN1K
import torch



rf = RF(
    MMDiT_for_IN1K(
        in_channels=4,
        out_channels=4,
        dim=2560,
        global_conddim=2560,
        n_layers=24,
        n_heads=8,
    ),
    True,
).cuda()

rf.load_state_dict(torch.load('/home/host/simo/ckpts/5b_2/model_57345/ema1.pt')) # checkpoint after getting high loss
rf.eval()

    

In [ ]:

conds = [142, 145, 933, 849, 94, 333, 980, 22, 77]

L = len(conds)
#conds = [1, ] * 4
conds = torch.tensor(conds).cuda()
init_noise = torch.randn(L, 4, 32, 32).cuda()
#conds = torch.randint(0, 999, (L,)).cuda()
STEPSIZE = 50
images = rf.sample_with_xps(init_noise, conds, null_cond = torch.tensor([1000] * L).cuda(), sample_steps = STEPSIZE, cfg = 7.0)

In [ ]:
from diffusers.image_processor import VaeImageProcessor
from diffusers.models import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae").to("cuda:0")

In [ ]:
images_for_grid = [[] for _ in range(L)]

for i, labidx in enumerate(conds[:L]):
    for t in range(STEPSIZE):
        x = vae.decode(images[t][i : i + 1].cuda()/0.13025).sample
        img = VaeImageProcessor().postprocess(
            image=x.detach(), do_denormalize=[True, True]
        )[0]
        images_for_grid[i].append(img)

In [ ]:
# plot 4 of them in grid
import PIL.Image as Image
grid_images = []
import math
sL = math.ceil(math.sqrt(L))
for t in range(STEPSIZE):
    
    grid = Image.new('RGB', (256*sL, 256*sL))
    for i in range(L):
        img = images_for_grid[i][t]
 
        grid.paste(img, (256 * (i % sL), 256 * (i // sL)))
    grid_images.append(grid)

# make it gif
# make last image lnger
grid_images += [grid_images[-1]] * 15
grid_images[-1]

In [ ]:
conds
142, 145, 933, 849, 

In [ ]:
grid_images[0].save('out_IN5.4b.gif', save_all=True, append_images=grid_images[1:], duration=150, loop=0)
